## Prepare data

In [ ]:
with open("./examples/data/text_forward.txt", "r+") as fr:
    lines = fr.readlines()

In [2]:
lines = [line.strip() for line in lines]

In [3]:
lines = [line.split() for line in lines]

In [4]:
lines = [line[::-1] for line in lines]

In [5]:
lines = [" ".join(line) for line in lines]

In [6]:
lines = lines[::-1]

In [7]:
lines = "\n".join(lines)

In [8]:
with open("./examples/data/text_backward.txt", "w+") as fw:
    fw.write(lines)

## Sanity Check

In [1]:
import torch
from newlm.lm.elmo.modeling_elmo.elmo_head import ELMOBertLMHeadModel
from newlm.lm.elmo.lm_builder import ELMOLMBuilder
from transformers import BertConfig

In [2]:
from newlm.utils.file_util import read_from_yaml
config_file = read_from_yaml('examples/configs/run.1-percent-bert-causal.yaml')

elmo_lm_builder = ELMOLMBuilder(
    model_config = config_file['lm']['model']['config'],
    tokenizer="bert-base-cased",
    model_type="bert-causal-elmo"
)

# model
config = BertConfig(**elmo_lm_builder.model_config)
model = ELMOBertLMHeadModel(config=config)

In [3]:
model.eval()
print("Model in eval mode for consistency")

Model in eval mode for consistency


In [4]:
%%capture

# dataset-forward
train_path = "./examples/data/text_forward.txt"
ds_f = elmo_lm_builder._get_dataset(train_path)

# dataset-backward
train_path = "./examples/data/text_backward.txt"
ds_b = elmo_lm_builder._get_dataset(train_path)

2021-11-10 17:03:11.685 | INFO     | newlm.lm.elmo.lm_builder:_get_dataset:142 - Constructing roBERTa style dataset
2021-11-10 17:03:12.892 | INFO     | newlm.lm.elmo.lm_builder:_get_dataset:142 - Constructing roBERTa style dataset


In [5]:
# trainer (for helper)
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir="tmpout",**config_file['lm']['hf_trainer']['args'])

trainer = Trainer(model=model, args=args, data_collator=elmo_lm_builder.data_collator,
    train_dataset=ds_f,
)
dl_f = trainer.get_train_dataloader() # Data Loader-forward

trainer = Trainer(model=model, args=args,data_collator=elmo_lm_builder.data_collator,
    train_dataset=ds_b,
)
dl_b = trainer.get_train_dataloader() # Data Loader-backward

max_steps is given, it will override any value given in num_train_epochs
max_steps is given, it will override any value given in num_train_epochs


### Compare text_forward and text_backward

In [6]:
batch_f = next(iter(dl_f))
batch_b = next(iter(dl_b))

batch_f['input_ids'].shape, batch_b['input_ids'].shape

(torch.Size([1, 271]), torch.Size([1, 271]))

In [15]:
tokens_f = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_f['input_ids'][0])
tokens_b = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_b['input_ids'][0])

import pandas as pd
pd.DataFrame({"forward": tokens_f, "backward": tokens_b})

,forward,backward
0,[CLS],[CLS]
1,R,.
2,##ø,periods
3,##d,republican
4,##berg,and
...,...,...
266,and,R
267,republican,##ø
268,periods,##d
269,.,##berg


Here we can see that the data is not completely flip when the tokenizer couldn't parse a single word into a single id

In [16]:
res = model(**batch_f)

l2r_loss tensor(10.5373, grad_fn=<NllLossBackward>)
r2l_loss tensor(10.5472, grad_fn=<NllLossBackward>)


In [17]:
res = model(**batch_b)

l2r_loss tensor(10.5231, grad_fn=<NllLossBackward>)
r2l_loss tensor(10.5489, grad_fn=<NllLossBackward>)


### From batch_forward compare Normal vs Rev

In [10]:
import torch

batch_f_input = torch.clone(batch_f['input_ids'])
batch_f_rev_input = torch.cat(
    (
        batch_f_input[0][0:1],
        torch.flip(batch_f_input[0][1:-1], [0]),
        batch_f_input[0][-1:]
    )
)
batch_f_rev_input = batch_f_rev_input.reshape(1,-1)

batch_f_labels = torch.clone(batch_f['labels'])
batch_f_rev_labels = torch.cat(
    (
        batch_f_labels[0][0:1],
        torch.flip(batch_f_labels[0][1:-1], [0]),
        batch_f_labels[0][-1:]
    )
)
batch_f_rev_labels = batch_f_rev_labels.reshape(1,-1)

batch_rev = batch_f.copy()
batch_rev['input_ids'] = batch_f_rev_input
batch_rev['labels'] = batch_f_rev_labels

In [14]:
tokens_f = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_f['input_ids'][0])
tokens_f_rev = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_rev['input_ids'][0])

import pandas as pd
pd.DataFrame({"forward": tokens_f, "reverse": tokens_f_rev})

,forward,reverse
0,[CLS],[CLS]
1,R,.
2,##ø,periods
3,##d,republican
4,##berg,and
...,...,...
266,and,##berg
267,republican,##d
268,periods,##ø
269,.,R


with the exception of [CLS] and [SEP], the data are completely flip

In [19]:
res = model(**batch_f)

l2r_loss tensor(10.5373, grad_fn=<NllLossBackward>)
r2l_loss tensor(10.5472, grad_fn=<NllLossBackward>)


In [18]:
res = model(**batch_rev)

l2r_loss tensor(10.5330, grad_fn=<NllLossBackward>)
r2l_loss tensor(10.5643, grad_fn=<NllLossBackward>)
